In [ ]:
# Example: My preferred approach
import os
# Settings -> Developer Settings -> Personal Access Tokens -> Token (classic)
os.environ['GITHUB_TOKEN'] = ""

GITHUB_USERNAME = "Codfishz"
REPO_NAME       = "ASR"
TOKEN = os.environ.get("GITHUB_TOKEN")
repo_url        = f"https://{TOKEN}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"
!git clone {repo_url}

Cloning into 'ASR'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 40 (delta 13), reused 32 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 10.81 MiB | 38.99 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [2]:
!cd {REPO_NAME} && git pull

Already up to date.


In [3]:
import os
os.chdir('ASR/Script')

In [4]:
%run "Datapreparation_YOLO.ipynb"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.2
    Uninstalling kaggle-1.7.4.2:
      Successfully uninstalled kaggle-1.7.4.2
Dataset URL: https://www.kaggle.com/datasets/nirmalsankalana/crop-pest-and-disease-detection
License(s): CC0-1.0
✅ Total images found in dataset: 25220
Images before filter: 25220


Saving train: 100%|██████████| 20176/20176 [00:30<00:00, 656.34it/s]



train split summary:
  Total images: 20109



Saving val: 100%|██████████| 2522/2522 [00:03<00:00, 654.48it/s]



val split summary:
  Total images: 2508



Saving test: 100%|██████████| 2522/2522 [00:03<00:00, 669.62it/s]


test split summary:
  Total images: 2509

  Images after filter : 25126



In [5]:
!pip install ultralytics
!pip install torchinfo

import ultralytics
ultralytics.checks()

Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 43.7/235.7 GB disk)


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from ultralytics import YOLO
print("GPU available:", torch.cuda.is_available())

GPU available: True


In [7]:
from ultralytics import YOLO
from torchinfo import summary

model = YOLO("yolo11n-cls.pt")
summary(model.model.cuda(), input_size=(1, 3, 640, 640))
net = model.model

100%|██████████| 5.52M/5.52M [00:00<00:00, 187MB/s]


In [23]:
ORIGINAL_DATASET_DIR = '/content/data_yolo'
ACTIVE_DATASET_DIR = '/content/data_active'

TRAIN_SUBDIR = 'train'
VAL_SUBDIR = 'val'
TEST_SUBDIR = 'test'
Num_Train = 20109
N_INITIAL = int(0.3 * Num_Train)
N_PER_PHASE = int(0.1 * Num_Train)
NUM_PHASES = 5

In [21]:
def setup_active_dataset():

    orig_train_dir = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR)
    active_train_dir = os.path.join(ACTIVE_DATASET_DIR, TRAIN_SUBDIR)
    os.makedirs(active_train_dir, exist_ok=True)

    for cls in os.listdir(orig_train_dir):
        cls_path = os.path.join(orig_train_dir, cls)
        if os.path.isdir(cls_path):
            os.makedirs(os.path.join(active_train_dir, cls), exist_ok=True)
    print(f"Complete creating train_dir {active_train_dir}")

    orig_val_dir = os.path.join(ORIGINAL_DATASET_DIR, VAL_SUBDIR)
    active_val_dir = os.path.join(ACTIVE_DATASET_DIR, VAL_SUBDIR)
    if os.path.exists(orig_val_dir):
        if os.path.exists(active_val_dir):
            shutil.rmtree(active_val_dir)
        shutil.copytree(orig_val_dir, active_val_dir)

    orig_test_dir = os.path.join(ORIGINAL_DATASET_DIR, TEST_SUBDIR)
    active_test_dir = os.path.join(ACTIVE_DATASET_DIR, TEST_SUBDIR)
    if os.path.exists(orig_test_dir):
        if os.path.exists(active_test_dir):
            shutil.rmtree(active_test_dir)
        shutil.copytree(orig_test_dir, active_test_dir)

In [10]:
def get_all_samples():

    samples = []
    orig_train_dir = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR)
    for cls in os.listdir(orig_train_dir):
        cls_path = os.path.join(orig_train_dir, cls)
        if os.path.isdir(cls_path):
            image_files = [f for f in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, f))]
            for f in image_files:
                samples.append((cls, f))
    return samples

In [11]:
import shutil
def copy_samples(sample_list):

    for cls, file_name in sample_list:
        src_path = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR, cls, file_name)
        dst_path = os.path.join(ACTIVE_DATASET_DIR, TRAIN_SUBDIR, cls, file_name)
        if not os.path.exists(dst_path):
            shutil.copy(src_path, dst_path)

In [12]:
import random
def stratified_sample(all_samples, n_initial):

    samples_by_class = {}
    for cls, filename in all_samples:
        samples_by_class.setdefault(cls, []).append((cls, filename))

    stratified = []
    for cls, samples in samples_by_class.items():
        stratified.append(random.choice(samples))
    remaining_count = n_initial - len(stratified)
    if remaining_count > 0:
        remaining_samples = list(set(all_samples) - set(stratified))
        additional_samples = random.sample(remaining_samples, remaining_count)
        stratified.extend(additional_samples)

    return stratified



In [20]:
print(ACTIVE_DATASET_DIR)

/content/data_active


In [24]:
setup_active_dataset()
all_samples = get_all_samples()

current_sample_list = stratified_sample(all_samples, N_INITIAL)
copy_samples(current_sample_list)

print("Initial Epoch")
results_train = model.train(data=ACTIVE_DATASET_DIR, epochs=1, imgsz=640)
remaining_samples = list(set(all_samples) - set(current_sample_list))

for phase in range(1, NUM_PHASES):
    n_to_add = min(N_PER_PHASE, len(remaining_samples))
    if n_to_add <= 0:
        print("No more samples")
        break

    new_samples = random.sample(remaining_samples, n_to_add)
    copy_samples(new_samples)

    current_sample_list.extend(new_samples)
    remaining_samples = list(set(remaining_samples) - set(new_samples))

    print(f"Phase {phase+1}：add {n_to_add} samples，total sample number: {len(current_sample_list)}。")

    results_train = model.train(data=ACTIVE_DATASET_DIR, epochs=1, imgsz=640, resume=True)
    print(f"Phase {phase+1} training completed")



Complete creating train_dir /content/data_active/train
Initial Epoch
Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=/content/data_active, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=

train: Scanning /content/data_active/train... 13133 images, 0 corrupt: 100%|██████████| 13133/13133 [00:00<?, ?it/s]
val: Scanning /content/data_active/val... 2508 images, 0 corrupt: 100%|██████████| 2508/2508 [00:00<00:00, 3523.47it/s]

val: WARNING ⚠️ /content/data_active/val/Tomato leaf curl/00037_Tomato_leaf_curl.jpg: corrupt JPEG restored and saved
val: New cache created: /content/data_active/val.cache


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000385, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/classify/train3
Starting training for 1 epochs...

      Epoch    GPU_mem       loss  Instances       Size


        1/1      1.93G      3.291         16        640:   1%|          | 8/821 [00:01<01:56,  6.96it/s]

...

train: New cache created: /content/data_active/train.cache


val: Scanning /content/data_active/val... 2508 images, 0 corrupt: 100%|██████████| 2508/2508 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/data_active/val/Tomato leaf curl/00037_Tomato_leaf_curl.jpg: corrupt JPEG restored and saved


WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in in the future.
optimizer: SGD(lr=0.2, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0001), 40 bias(decay=0.0)


AssertionError: yolo11n-cls.pt training to 200 epochs is finished, nothing to resume.
Start a new training without resuming, i.e. 'yolo train model=yolo11n-cls.pt'

In [ ]:
results_train = model.train(data='/content/data_yolo', epochs=10, imgsz=640)

Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=/content/data_yolo, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=No

100%|██████████| 5.35M/5.35M [00:00<00:00, 343MB/s]


AMP: checks passed ✅


train: Scanning /content/data_yolo/train... 20109 images, 0 corrupt: 100%|██████████| 20109/20109 [00:06<00:00, 3193.17it/s]

train: WARNING ⚠️ /content/data_yolo/train/Maize leaf beetle/00147_Maize_leaf_beetle.jpg: corrupt JPEG restored and saved


...

      10/10      2.42G     0.2649         13        640: 100%|██████████| 1257/1257 [02:44<00:00,  7.64it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 79/79 [00:10<00:00,  7.54it/s]

                   all      0.904      0.999



10 epochs completed in 0.491 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.2MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.2MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-cls summary (fused): 47 layers, 1,554,206 parameters, 0 gradients, 3.2 GFLOPs
train: /content/data_yolo/train... found 20109 images in 22 classes ✅ 
val: /content/data_yolo/val... found 2508 images in 22 classes ✅ 
test: /content/data_yolo/test... found 2509 images in 22 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 79/79 [00:10<00:00,  7.84it/s]


                   all      0.906      0.999
Speed: 0.7ms preprocess, 1.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


# 上面是现在能跑的内容

In [ ]:
# results_val = model.val()

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()


In [ ]:
  !ls

Dataloader_FeatureExtraction.ipynb  runs	    yolov8n.pt
datasets			    yolo11n-cls.pt
mc_dropout.ipynb		    YOLO.ipynb


In [ ]:
def active_learning_selector(model, dataset, candidate_indices, num_select):


    return np.random.choice(candidate_indices, num_select, replace=False).tolist()

In [ ]:
print(len(dataset))

24453


In [ ]:
# https://docs.ultralytics.com/zh/datasets/classify/#folder-structure-example

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    model.train()
    running_loss = 0.0

    for batch_idx, (images, targets) in enumerate(dataloader):
        images = images.cuda() if torch.cuda.is_available() else images
        targets = [{k: v.cuda() if torch.cuda.is_available() else v for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        pred = model(images)  # Forward pass
        loss, loss_items = model.compute_loss(pred, targets)  # YOLO 自带损失函数
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (batch_idx + 1) % 10 == 0:
            print(f"Batch {batch_idx+1}: Loss = {loss.item():.4f}")

    print(f"Epoch {epoch+1} finished, avg loss: {running_loss / (batch_idx+1):.4f}")

    # Active Learning 部分：选择一部分数据重新标注或训练
    candidate_indices = list(range(len(dataset)))
    new_indices = active_learning_selector(model, dataset, candidate_indices, num_select=int(len(candidate_indices) * 0.5))

    active_sampler.update_indices(new_indices)

    dataloader = DataLoader(dataset, batch_size=256, sampler=active_sampler, num_workers=0, pin_memory=True)


Epoch 1/10
Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=../datasets/imagenet, epochs=100, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True,

RuntimeError: Dataset '../datasets/imagenet' error ❌ [Errno 2] No such file or directory: '/content/ASR/Script/datasets/imagenet/train'

In [ ]:

num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    net.train()
    running_loss = 0.0
    for batch_idx, (images, labels) in enumerate(dataloader):
        images = images.cuda() if torch.cuda.is_available() else images
        labels = labels.cuda() if torch.cuda.is_available() else labels

        optimizer.zero_grad()
        # outputs = net(images)
        # loss = criterion(outputs, labels)
        # loss.backward()
        # optimizer.step()

        pred = model(images)
        loss, loss_items = compute_loss(pred, targets)  # 自带的 loss 函数
        loss.backward()
        optimizer.step()


        running_loss += loss.item()
        if (batch_idx + 1) % 10 == 0:
            print(f"Batch {batch_idx+1}: Loss = {loss.item():.4f}")

    print(f"Epoch {epoch+1} finished, avg loss: {running_loss / (batch_idx+1):.4f}")

    candidate_indices = list(range(len(dataset)))
    new_indices = active_learning_selector(net, dataset, candidate_indices, num_select=int(len(candidate_indices)*0.5))

    active_sampler.update_indices(new_indices)

    dataloader = DataLoader(dataset, batch_size=256, sampler=active_sampler, num_workers=0, pin_memory=True)

Epoch 1/10


TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not list

In [ ]:
# results_pred = model.predict(source="/content/datasets/coco128/images/val2017", conf=0.25)

# for result in results_pred:
#     result.show()


# print("Result Train：", results_train)
# print("Result Validation：", results_val)

FileNotFoundError: /content/datasets/coco128/images/val2017 does not exist